In [6]:
#First tradingEnv class
#todo add transaction cost
# add MACD indicator
# add RSI indicator
# add SMA indicator
# add EMA indicator
# meerdere crypto's toevoegen ( momenteel alleen BTC)
import numpy as np
import pandas as pd
class TradingEnv():
    def __init__(self, initial_investment=20000, Terminal_state= 2000):
        #investment
        self.initial_investment = initial_investment #initial investment 
        self.Terminal_state = Terminal_state  #terminal state, when the agent reach this state or below, the episode is over
        self.current_investment = initial_investment #current investment in a certain state
        self.non_invested = initial_investment #non invested money in a certain state
        self.invested = 0
        self.worth_history = [] #worth history in a certain episode
        # self.invested = [{'amount': 0, 'price_investment': 0, 'timestamp' : UNIX timestamp of investment}] #invested money 
        self.invested_amount = 0
        
        #rewards
        self.current_reward = 0 #current reward in a certain state
        self.total_reward = 0   #total reward in a certain episode
        self.reward_history = [] #reward history in a certain episode
        self.iterations = 0

        #states
        self.current_price = [0,0,0,0] #current price in a certain state [open,high,low,close]
        self.prev_price = np.zeros(20) #previous 20 prices to keep in memory to the state values
        self.done = False #if the episode is over
        self.current_unix_time = 0 #current unix time in a certain state
        self.macd = 0 #macd indicator
        
        
        #actions
        self.action = 0 #chosen action in a certain state
        self.action_history = [] #action history in a certain episode
        self.action_space = np.array([0,1,2]) #action space, 0 = hold, 1 = buy, 2 = sell first in the array 

        #Current state
        # self.current_state = self.get_current_state() #current state
        self.csv_to_dataframe() #convert csv's to one dataframe
        self.csv_index = 0 #index of the csv dataframe
        self.demo = False

    def get_current_state(self):
        #return current state price, and invested/non invested values as state
        return [self.current_price[0],self.current_price[1],self.current_price[2],self.current_price[3], self.invested, self.non_invested, self.invested_amount, self.current_unix_time, self.macd,self.sma10,self.sma20, self.counter, self.lastBuy, self.lastSold, self.lastBuyPrice, self.lastSoldPrice]
    # def calculate_macd(self):
    #     #calculate macd indicator
    #     print("macd")
    #     # print(self.df.iloc[self.csv_index - 12 : self.csv_index, 3])
    #     # print(self.df.iloc[self.csv_index - 12 : self.csv_index, 3].ewm(span=12, adjust=False).mean())
    #     macd = self.df.iloc[self.csv_index - 500 : self.csv_index, 3].ewm(span=500, adjust=False).mean()
    #     macd = macd.tolist()
    #     print('res',macd)
    #     # print('res',type(macd.tolist()))
    #     print("average macd", sum(macd) / len(macd))

        return macd
    def reset(self, demo=False):
        # self.lastBuy, self.lastSold, self.lastBuyPrice, self.lastSoldPrice
        self.lastBuy = 0
        self.lastSold = 0
        self.lastBuyPrice = 0
        self.lastSoldPrice = 0
        
        self.counter = 0
        #reset the environment
        self.worth_history = []
        self.current_investment = self.initial_investment
        self.current_reward = 0
        self.total_reward = 0
        self.reward_history = []
        self.current_price = [0,0,0,0] #current price in a certain state [open,high,low,close]
        self.invested_amount = 0
        self.non_invested = self.initial_investment #non invested money in a certain state
        self.invested = 0
        self.worth_history = [] #worth history in a certain episode

        self.prev_price = np.zeros(20)
        self.action = 0
        self.action_history = []
        self.done = False
        self.current_unix_time = 0 #current unix time in a certain state
        self.csv_index = 0 #index of the csv dataframe
        self.demo = demo

        #begin somewhere random in csv data 
        
        if demo == False:
            #last 6months as test set
            self.csv_index = np.random.randint(0, (len(self.df)/2) + 40000)
        else:
            self.csv_index = len(self.df) - 262975
        #get row csv_index from self.df
        self.current_price = self.df.iloc[self.csv_index, 3:7].values
        self.current_unix_time = self.df.iloc[self.csv_index, 0]
        
        self.macd =  self.df.iloc[self.csv_index, 10]
        self.sma20 =  self.df.iloc[self.csv_index, 9]
        self.sma10 =  self.df.iloc[self.csv_index, 11]






        return self.get_current_state(), self.done

    def step(self, action, interval = 1):
        self.counter +=1
        self.iterations += 1
        self.action = action
        self.action_history.append(self.action)
        
        #calculate current reward
        self.current_reward = self.get_reward()
        self.done = self.is_done()
        #if the episode is over, give a big negative reward
        if self.done == True:
            self.current.reward - 1000000

        self.total_reward += self.current_reward
        self.reward_history.append(self.current_reward)

        #get next line of data in csv
        if self.csv_index < len(self.df) - 1:
            self.csv_index += interval
            self.current_price = self.df.iloc[self.csv_index, 3:7].values
            self.current_unix_time = self.df.iloc[self.csv_index, 0]
            self.macd =  self.df.iloc[self.csv_index, 10]
            self.sma20 =  self.df.iloc[self.csv_index, 9]
            self.sma10 =  self.df.iloc[self.csv_index, 11]

        else:
            self.done = True
        if self.csv_index > len(self.df) - 262975 and self.demo == False:
            self.done = True
        
        return self.get_current_state(), self.current_reward, self.done
        
    def get_reward(self):
        #reward function
        if self.action == 0:
            #hold do nothing
            reward = .1
        elif self.action == 1:
    
            #buy = invest 1/10 of non_invested money
            to_invest = self.non_invested / 10
            self.non_invested -= to_invest
            amount = to_invest / self.current_price[0]
            self.invested += to_invest
            self.invested_amount += amount
            self.lastBuy = self.counter
            self.lastBuyPrice = self.current_price[0]
            reward = .1
        elif self.action == 2:
        
            amount = self.invested_amount
            price_investment = self.invested
            self.non_invested += amount * self.current_price[0]
            self.invested_amount = 0
            self.invested = 0
            self.lastSold = self.counter
            self.lastSoldPrice = self.current_price[0]

            reward = ((amount * self.current_price[0]) - price_investment)**3
           
        
        self.worth_history.append(self.calc_current_worth())
        if reward > 0: 
            return (reward**2) * ((self.calc_current_worth()/self.initial_investment)**2)
        else:
            return (reward**3) * ((self.calc_current_worth()/self.initial_investment)**2)



    def is_done(self):
        #terminal state
        if self.calc_current_worth() <= self.Terminal_state:
            return True
        else:
            return False

    def calc_current_worth(self):
        #calculate current worth
        return self.invested_amount * self.current_price[0] + self.non_invested

        
    def csv_to_dataframe(self):
        #convert csv's to one dataframe
        df2017 = pd.read_csv('../data/BTC-2017min.csv')
        df2018 = pd.read_csv('../data/BTC-2018min.csv')
        df2019 = pd.read_csv('../data/BTC-2019min.csv')
        df2020 = pd.read_csv('../data/BTC-2020min.csv')
        df2021 = pd.read_csv('../data/BTC-2021min.csv')

        #concat all dataframes
        df = pd.concat([df2017, df2018, df2019, df2020, df2021], ignore_index=True)
        #add new column named macd, that is the mean of the last 12 open values
        df['sma40000'] = df['open'].rolling(window=40000).mean()
        self.df = df
        df['macd20000'] = df['open'].rolling(window=20000).mean()**2
        self.df = df
        df['sma10000'] = df['open'].rolling(window=20000).mean()
        df.replace([np.inf, -np.inf], np.nan, inplace=True) 
        df.dropna(inplace=True) 
        self.df = df
        #len = 2675301

In [13]:
class GridTrading:
    def __init__(self, current_price) -> None:
        self.line_count = 5
        self.grid_space = 100
        self.current_price = current_price
    
    def calculate_grid_lines(self):
        line_count = self.line_count
        grid_space = self.grid_space
        price = self.current_price
        
        stop_loss = price - ((line_count + 1) * grid_space)
        take_profit = price + ((line_count + 1) * grid_space)
        buy_lines = []
        sell_lines = []
     
        for i in range(line_count):
             buy_lines.append(price - ((line_count * grid_space)))
             sell_lines.append(price + ((line_count * grid_space)))
         
        return stop_loss, take_profit, buy_lines, sell_lines
    
    # risk no more than 10 percent of our account on a single trade
    # when the stoploss is triggered we should only lose 10 percent
    def calculate_position_size(self, account_size):
        line_count = self.line_count
        grid_space = self.grid_space
        
        risk = account_size * .1
        tick_value = 1
        ticks_at_risk = (line_count + 1) * grid_space
        position_size = risk / (ticks_at_risk * tick_value)
        position_size = round(position_size, 2)
        return position_size

    

env = TradingEnv()
iterations = 1000
sold = []
bought = []
worth = []
price = []
state, done = env.reset()
gridTrading = GridTrading(env.current_price[0])
counter = 0
for i in range(3):
    #Buy 3 times at start to have some amount to work with
    state, reward, done = env.step(1)
    counter += 1
    print(env.invested, reward)
print("invested: ", env.invested, "non_invested: ", env.non_invested, "worth: ", env.calc_current_worth())
pos_size = gridTrading.calculate_position_size(env.non_invested)
print("position size: ", pos_size)

stop_loss, take_profit, buy_lines, sell_lines = gridTrading.calculate_grid_lines()

for i in range(iterations):
    if state[0] in buy_lines:
        print("buy")
        state, reward, done = env.step(1)
        bought.append(counter)
        # print(env.invested, reward)
    elif state[0] in sell_lines:
        print("sell")
        sold.append(counter)
        state, reward, done = env.step(3)
        # print(env.invested, reward)
    elif take_profit == state[0]:
        print("sell")
        # sold.append(counter)

        state, reward, done = env.step(2)
        print(env.invested, reward)
    elif stop_loss == state[0]:
        print("sell")
        sold.append(counter)

        state, reward, done = env.step(2)
        # print(env.invested, reward)
    else:
        state, reward, done = env.step(0)
        # print(env.invested, reward)

    counter += 1

#
print(buy_lines)
     


2000.0 0.010000000000000002
3800.0 0.010002724981253113
5420.0 0.010026205011426747
invested:  5420.0 non_invested:  14580.0 worth:  20032.38127703651
position size:  2.43
[6840.0, 6840.0, 6840.0, 6840.0, 6840.0]
